In [1]:
import numpy as np
import collections
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [2]:
edge_filename = './data/email-Eu-core.txt'
output_filename = './testout3.txt'
k = 42

In [3]:
edge_file = open(edge_filename, 'r')

In [4]:
edges = []
while True:
    line = edge_file.readline().split()
    if not line:
        break
    edges.append(line)
edge_file.close()

In [5]:
d = collections.defaultdict(set)
for edge in edges:
    if edge[0] == edge[1]:
        if edge[0] not in d:
            d[edge[0]] = set()
        if edge[1] not in d:
            d[edge[1]] = set()
        continue
    if edge[0] in d:
        d[edge[0]].add(edge[1])
    else:
        d[edge[0]] = {edge[1]}
    if edge[1] in d:
        d[edge[1]].add(edge[0])
    else:
        d[edge[1]] = {edge[0]}

In [6]:
nodes = [k for k in d.keys()]
node_id_idx = {}
node_idx_id = {}
for i in range(len(nodes)):
    node_id_idx[nodes[i]] = i
    node_idx_id[i] = nodes[i]

In [7]:
node_id_idx['1004']

1004

In [8]:
len(nodes)

1005

In [9]:
L = np.zeros((len(nodes), len(nodes)))

In [10]:
for nid in d:
    idx = node_id_idx[nid]
    L[idx][idx] = len(d[nid])
    for i in d[nid]:
        cur_idx = node_id_idx[i]
        L[idx][cur_idx] = -1

In [11]:
L

array([[42., -1.,  0., ...,  0.,  0.,  0.],
       [-1., 50.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 93., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [12]:
E = np.linalg.eig(L)

In [13]:
E_val = []
for i, val in enumerate(E[0]):
    E_val.append((i, np.real(val)))

In [14]:
E_val

[(0, 346.08069671540494),
 (1, 233.22632228540982),
 (2, 232.17670516297292),
 (3, 220.2685680424513),
 (4, 217.21844468206965),
 (5, 215.12690419246852),
 (6, 184.37376570774663),
 (7, 179.43013191528615),
 (8, 176.48496476183777),
 (9, 172.43506870066938),
 (10, 170.43904079515096),
 (11, 169.55903087016944),
 (12, 166.97162504613854),
 (13, 163.28967417827295),
 (14, 158.455429287734),
 (15, 153.43613805384942),
 (16, 145.95060226892343),
 (17, 145.2633971773136),
 (18, 142.45972966576986),
 (19, 141.92938586609273),
 (20, 141.73314953139018),
 (21, 140.9881628445993),
 (22, 140.06757987616484),
 (23, 139.07875357803476),
 (24, 138.63812055814304),
 (25, 137.72722423135764),
 (26, 138.2877375781068),
 (27, 136.54729415976752),
 (28, 134.4653842527019),
 (29, 133.53354577918088),
 (30, 131.52714095284125),
 (31, 130.62759468318325),
 (32, 130.3264616934867),
 (33, 128.88158225628993),
 (34, 127.71031591277551),
 (35, 124.90438225204868),
 (36, 124.05307645245651),
 (37, 123.443035372

In [15]:
E_val_sorted = sorted(E_val, key=lambda x: x[1])
E_val_sorted

[(986, 0.0),
 (987, 0.0),
 (988, 0.0),
 (989, 0.0),
 (990, 0.0),
 (991, 0.0),
 (992, 0.0),
 (993, 0.0),
 (994, 0.0),
 (995, 0.0),
 (996, 0.0),
 (997, 0.0),
 (998, 0.0),
 (999, 0.0),
 (1000, 0.0),
 (1001, 0.0),
 (1002, 0.0),
 (1003, 0.0),
 (1004, 0.0),
 (80, 1.5649065982478054e-13),
 (516, 0.5641205159799334),
 (410, 0.6938578656934467),
 (517, 0.7062460730565638),
 (738, 0.8575129636529909),
 (745, 0.875978590127389),
 (776, 0.896614252891651),
 (746, 0.8994084285563189),
 (778, 0.9090733344486112),
 (794, 0.9134074893346585),
 (795, 0.9268517466438418),
 (824, 0.939936781972903),
 (882, 0.9415555741056971),
 (884, 0.9503566696812759),
 (885, 0.9536662312825476),
 (886, 0.9571885074773289),
 (887, 0.9593261811483256),
 (889, 0.9613979515080414),
 (890, 0.9617843385815419),
 (891, 0.9625025808161323),
 (888, 0.963443870829091),
 (911, 0.964434376721056),
 (912, 0.9648892391524764),
 (915, 0.9663188364741628),
 (916, 0.9665989846226852),
 (914, 0.9680648507786028),
 (913, 0.9687843205982

In [16]:
len(E[1][:,6])

1005

In [17]:
E[0][890]

(0.9617843385815419+0j)

In [18]:
original_cnt = cnt = 3 # chose #cnt of eigenvalue as node embedding
embed_i = []
for i, eig_val in E_val_sorted:
    if cnt == 0:
        break
    elif eig_val > 1e-5:
        embed_i.append(i)
        cnt -= 1

In [19]:
embed_i

[516, 410, 517]

In [20]:
T = np.zeros((len(nodes), original_cnt))
for i,idx in enumerate(embed_i):
    j = 0
    for val in E[1][:,idx]:
        T[j][i] = np.real(val)
        j += 1

In [21]:
T

array([[-0.00223817,  0.00108025, -0.00015238],
       [-0.00236574,  0.00097304, -0.00016048],
       [-0.0025915 ,  0.00093726, -0.0001213 ],
       ...,
       [-0.00572539,  0.00380569, -0.0003596 ],
       [-0.00583494,  0.00379419, -0.00053634],
       [-0.00607037,  0.00300981, -0.0004181 ]])

In [29]:
list(T)

[array([-0.00223817,  0.00108025, -0.00015238]),
 array([-0.00236574,  0.00097304, -0.00016048]),
 array([-0.0025915 ,  0.00093726, -0.0001213 ]),
 array([-0.00255526,  0.00088849, -0.0001258 ]),
 array([-2.59484527e-03, -2.23868811e-03, -9.48894147e-05]),
 array([-2.03595903e-03,  1.06397081e-03,  8.61663207e-05]),
 array([-0.00241247,  0.00099562, -0.00012858]),
 array([-0.0025778 ,  0.0011309 , -0.00014321]),
 array([-0.00259605,  0.00113714, -0.0001426 ]),
 array([-0.00266808,  0.00117726, -0.00015151]),
 array([-0.00247337,  0.0010783 , -0.00011049]),
 array([-0.00255292,  0.00110337, -0.00014272]),
 array([-0.00256999,  0.0011547 , -0.00018245]),
 array([-2.25412138e-03,  1.00276891e-03, -9.88783843e-05]),
 array([-0.00146101,  0.0010208 , -0.0008906 ]),
 array([-0.00255627,  0.00107368, -0.00013378]),
 array([-0.00243002,  0.00096781, -0.00013252]),
 array([-0.00232825,  0.00099893, -0.00015085]),
 array([-0.00066104,  0.00103833, -0.00103744]),
 array([-0.00252504,  0.00109243,

In [22]:
kmeans = KMeans(n_clusters=k, random_state=0).fit(T)

In [23]:
res = kmeans.labels_
res

array([23, 23, 39, ..., 38, 38, 25])

In [24]:
list(res)

[23,
 23,
 39,
 39,
 16,
 15,
 37,
 37,
 0,
 0,
 37,
 37,
 37,
 23,
 28,
 37,
 37,
 23,
 21,
 37,
 37,
 37,
 37,
 23,
 23,
 15,
 15,
 15,
 23,
 23,
 23,
 23,
 23,
 15,
 23,
 23,
 23,
 15,
 15,
 15,
 15,
 15,
 23,
 0,
 37,
 37,
 37,
 15,
 15,
 37,
 37,
 23,
 37,
 23,
 39,
 39,
 39,
 39,
 39,
 39,
 15,
 15,
 23,
 39,
 23,
 39,
 37,
 37,
 37,
 37,
 37,
 37,
 17,
 37,
 23,
 23,
 15,
 37,
 37,
 23,
 37,
 37,
 23,
 23,
 37,
 15,
 15,
 37,
 0,
 37,
 37,
 37,
 37,
 23,
 15,
 23,
 23,
 0,
 37,
 37,
 37,
 37,
 39,
 15,
 15,
 23,
 37,
 37,
 37,
 0,
 0,
 0,
 37,
 15,
 23,
 23,
 15,
 37,
 37,
 23,
 23,
 37,
 28,
 23,
 37,
 37,
 39,
 37,
 23,
 23,
 23,
 39,
 37,
 23,
 0,
 15,
 23,
 39,
 39,
 37,
 37,
 37,
 23,
 23,
 37,
 39,
 23,
 23,
 15,
 28,
 15,
 15,
 23,
 37,
 37,
 23,
 18,
 28,
 37,
 37,
 23,
 37,
 37,
 37,
 37,
 23,
 23,
 15,
 23,
 23,
 23,
 15,
 23,
 37,
 39,
 37,
 15,
 36,
 28,
 28,
 28,
 15,
 15,
 23,
 37,
 0,
 37,
 37,
 37,
 37,
 37,
 36,
 39,
 39,
 39,
 39,
 23,
 23,
 23,
 23,
 15,
 23,


In [25]:
res_file = open('./data/email-Eu-core-department-labels.txt', 'r')

In [26]:
truth = []
while True:
    line = res_file.readline().split()
    if not line:
        break
    truth.append(int(line[1]))
res_file.close()

In [27]:
test = np.asarray(truth)
test

array([ 1,  1, 21, ...,  1,  6, 22])

In [28]:
adjusted_rand_score(res, test)

0.0904035358612251